In [49]:
import pandas as pd


df = pd.read_csv('./LaptrackPhase3.csv')
df.shape

(4551, 22)

In [50]:
df.columns

Index(['Brand', 'Laptop_Model_Name', 'Laptop_Model_Number', 'Processor_Brand',
       'Processor_Model', 'Storage_Type', 'Operating_System',
       'Display_Resolution', 'Extracted_Rating', 'Battery_Life(Hours_Upto)',
       'Price', 'Stock', 'Time_Of_Extraction', 'URL', 'Source',
       'Storage_Capacity(GB)', 'Display_Size(Inches)', 'RAM(GB)',
       'No_Of_Reviews', 'Laptop_Dimensions', 'Laptop_Weight(Pounds)',
       'image_src'],
      dtype='object')

In [51]:
df['Processor_Brand'].isnull().sum()

np.int64(4)

In [3]:
df['Processor_Model'].unique()

array(['Core i5', 'Core i7', 'Celeron N', 'AMD Ryzen 5 5500U', 'Ryzen 3',
       'Intel Mobile CPU', 'Ryzen 5', 'Celeron N4020', 'Core i3',
       'Celeron', 'Cortex', 'Apple M1', 'Ryzen 7', 'A13', 'Celeron P4500',
       'Ryzen 9', 'AMD Ryzen 7', 'Pentium N5000', 'Core i9',
       'Mediatek Mt8173C', 'AMD R Series', 'Celeron N4000',
       'Ryzen 5 4500U', 'Pentium', 'Celeron N3350', 'Apple M2', 'AMD A4',
       'Apple M1 Pro', 'AMD A Series', 'MediaTek MT8183', 'Others',
       'Athlon', 'Pentium Other', 'Celeron N3060', 'Ryzen 3 3200U',
       'Intel Celeron D', 'Celeron 5205U', 'Intel Pentium 4',
       'Ryzen 5 4600H', '1', 'Ryzen 3 3250U', 'Celeron N3450', 'H8S',
       'Core M 7Y30', 'AMD Ryzen 7 5700U', 'Intel Core Ultra 7',
       'AMD Ryzen 5 3450U', 'Intel N100', '68LC040', 'Ryzen 3 3350U',
       'Intel Core 7', 'Intel Processor N100', 'Intel Core Ultra 5',
       '8032', 'Celeron N2840', 'Intel Core Ultra 9', 'Intel PDC G2030',
       'Snapdragon', 'Celeron 330', 'Mobile I

In [53]:
len(df['Processor_Model'].unique())

198

In [52]:
df['Processor_Model'].value_counts()

Processor_Model
Core i5                                       924
Core i7                                       601
Core i3                                       404
Celeron                                       170
Intel Mobile CPU                              144
Ryzen 5                                       121
Ryzen 5 Hexa Core                             112
Celeron N                                     110
Ryzen 7                                        98
Core i9                                        96
Celeron N3350                                  76
Celeron Dual Core                              68
Intel 8th Generation Core i5                   67
Intel Core Ultra 7 Series 1                    65
Pentium                                        60
Intel 13th Generation Core i7                  52
Pentium N5000                                  40
Intel Core Ultra 5 Series 1                    40
AMD Ryzen 7                                    34
Intel Core Ultra 9 Series 1       

In [54]:
import re
# Pattern to match Apple processors and their variants (case-insensitive)
apple_pattern = r'\b(M\d+)(?:\s+(Pro|Max|Ultra|Plus))?\b'

# Function to normalize Apple processor models
def normalize_apple_processors(processor):
    # Convert the processor name to uppercase for consistent matching
    apple_pattern = r'\b(M\d+)(?:\s+(Pro|Max|Ultra|Plus))?\b'
    processor = processor.strip()  # Ensure the model is uppercase
    match = re.search(apple_pattern, processor, re.IGNORECASE)
    if match:
        base_model = match.group(1).upper()  # E.g., 'M1', 'M2', 'M3'
        variant = match.group(2)     # E.g., 'Pro', 'Max', 'Ultra'
        if variant:
            normalized_processor = f"{base_model} {variant}"  # Combine model and variant
        else:
            normalized_processor = base_model  # Just the model if no variant
        return (normalized_processor,True)
    else:
        return (processor,False)  # If no match, keep as is

# Function to normalize processor names
def normalize_processor(model):
    # Handle common patterns and normalize
    if pd.isna(model) or model == 'Unknown':
        return None

    # First, check if it's an Apple processor
    apple_normalized,status = normalize_apple_processors(model)
    if status:
        return apple_normalized

    # AMD Ryzen series
    amd_ryzen = re.search(r'(AMD )?(Ryzen \d)', model, re.IGNORECASE)
    if amd_ryzen:
        return amd_ryzen.group(2)

    # AMD Athlon or A-series
    amd_other = re.search(r'AMD (Athlon|A Series)', model, re.IGNORECASE)
    if amd_other:
        return 'AMD ' + amd_other.group(1)

    # Intel Core series
    intel_core = re.search(r'(Intel )?(Core i[3579])', model, re.IGNORECASE)
    if intel_core:
        # Normalize the 'Core iX' to 'Core iX' with lowercase 'i' for consistency
        core_model = intel_core.group(2).lower().capitalize()  # Converts 'Core I7' to 'Core i7'
        return core_model
    else:
        return model  # If no match, return the original model

    # # Celeron, Pentium, and Xeon series
    # if 'Celeron' in model:
    #     return 'Celeron'
    # if 'Pentium' in model:
    #     return 'Pentium'
    # if 'Xeon' in model:
    #     return 'Xeon'
    
    # # ARM-based processors
    # if 'ARM' in model or 'Cortex' in model:
    #     return 'ARM'
    
    # # Snapdragon and MediaTek
    # if 'Snapdragon' in model:
    #     return 'Snapdragon'
    # if 'MediaTek' in model:
    #     return 'MediaTek'
    
    # Others that might need further inspection
    return model

df = df.copy()
# Apply the normalization function to the DataFrame
df['Processor_Model'] = df['Processor_Model'].apply(normalize_processor)
df['Processor_Model'].unique()

array(['Core i5', 'Core i7', 'Celeron N', 'Ryzen 5', 'Ryzen 3',
       'Intel Mobile CPU', 'Celeron N4020', 'Core i3', 'Celeron',
       'Cortex', 'M1', 'Ryzen 7', 'A13', 'Celeron P4500', 'Ryzen 9',
       'Pentium N5000', 'Core i9', 'Mediatek Mt8173C', 'AMD R Series',
       'Celeron N4000', 'Pentium', 'Celeron N3350', 'M2', 'AMD A4',
       'M1 Pro', 'AMD A Series', 'MediaTek MT8183', 'Others', 'Athlon',
       'Pentium Other', 'Celeron N3060', 'Intel Celeron D',
       'Celeron 5205U', 'Intel Pentium 4', '1', 'Celeron N3450', 'H8S',
       'Core M 7Y30', 'Intel Core Ultra 7', 'Intel N100', '68LC040',
       'Intel Core 7', 'Intel Processor N100', 'Intel Core Ultra 5',
       '8032', 'Celeron N2840', 'Intel Core Ultra 9', 'Intel PDC G2030',
       'Snapdragon', 'Celeron 330', 'Mobile Intel Celeron Processor',
       'Mediatek 8317 Dual Core', 'Pentium D', 'Celeron Dual Core',
       'MT8183', 'Core Ultra 5', 'Celeron Quad Core', 'Core Ultra 7',
       'Ryzen Z1 Octa Core', 'M3', 'Ath

In [57]:
sorted(df['Processor_Model'].unique())

['1',
 '68LC040',
 '8032',
 'A13',
 'AMD A Series',
 'AMD A-Series A6',
 'AMD A4',
 'AMD R Series',
 'AMD Ryzen AI 300 Series',
 'APU Quad Core A12',
 'Athlon',
 'Athlon Dual Core',
 'Celeron',
 'Celeron 330',
 'Celeron 5205U',
 'Celeron Dual Core',
 'Celeron N',
 'Celeron N2840',
 'Celeron N3060',
 'Celeron N3350',
 'Celeron N3450',
 'Celeron N4000',
 'Celeron N4020',
 'Celeron P4500',
 'Celeron Quad Core',
 'Core 5',
 'Core 7',
 'Core M 7Y30',
 'Core N',
 'Core Ultra 5',
 'Core Ultra 7',
 'Core Ultra 9',
 'Core i3',
 'Core i5',
 'Core i7',
 'Core i9',
 'Cortex',
 'Dual Core',
 'H8S',
 'Intel Atom',
 'Intel Celeron',
 'Intel Celeron D',
 'Intel Core 2 Duo',
 'Intel Core 3 Series 1',
 'Intel Core 5 Series 1',
 'Intel Core 7',
 'Intel Core 7 Series 1',
 'Intel Core M',
 'Intel Core Ultra 5',
 'Intel Core Ultra 5 Series 1',
 'Intel Core Ultra 5 Series 2',
 'Intel Core Ultra 7',
 'Intel Core Ultra 7 Series 1',
 'Intel Core Ultra 7 Series 1 Evo Edition',
 'Intel Core Ultra 7 Series 2',
 'I

In [58]:
df['Processor_Model'].value_counts()

Processor_Model
Core i5                                    1162
Core i7                                     801
Core i3                                     429
Ryzen 5                                     334
Ryzen 7                                     208
Celeron                                     170
Core i9                                     149
Intel Mobile CPU                            144
Celeron N                                   110
Celeron N3350                                76
Celeron Dual Core                            68
Intel Core Ultra 7 Series 1                  65
Pentium                                      60
Ryzen 3                                      59
Intel Core Ultra 5 Series 1                  40
Pentium N5000                                40
M3                                           37
M2                                           37
Ryzen 9                                      36
Intel Core Ultra 9 Series 1                  33
Celeron N4020           

In [13]:
# Set the display options
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('future.no_silent_downcasting', True)
sorted(df['Processor_Model'].unique())

['1',
 '68LC040',
 '8032',
 'A13',
 'AMD A Series',
 'AMD A-Series A6',
 'AMD A4',
 'AMD R Series',
 'AMD Ryzen 3 7000 Series',
 'AMD Ryzen 5 3000 Series',
 'AMD Ryzen 5 3000 series PRO',
 'AMD Ryzen 5 3450U',
 'AMD Ryzen 5 4000 Series',
 'AMD Ryzen 5 5000 Series',
 'AMD Ryzen 5 5000 Series PRO',
 'AMD Ryzen 5 5500U',
 'AMD Ryzen 5 7000 Series',
 'AMD Ryzen 5 8000 Series',
 'AMD Ryzen 5 Microsoft Surface Edition',
 'AMD Ryzen 7',
 'AMD Ryzen 7 2000 Series',
 'AMD Ryzen 7 3000 Series',
 'AMD Ryzen 7 4000 Series PRO',
 'AMD Ryzen 7 5000 Series',
 'AMD Ryzen 7 5000 Series PRO',
 'AMD Ryzen 7 5700U',
 'AMD Ryzen 7 7000 Series',
 'AMD Ryzen 7 8000 Series',
 'AMD Ryzen 7 8000 Series PRO',
 'AMD Ryzen 7 Microsoft Surface Edition',
 'AMD Ryzen 9 5000 Series',
 'AMD Ryzen 9 6000 Series',
 'AMD Ryzen 9 7000 Series',
 'AMD Ryzen 9 8000 Series',
 'AMD Ryzen AI 300 Series',
 'APU Quad Core A12',
 'Apple M1',
 'Apple M1 Max',
 'Apple M1 Pro',
 'Apple M2',
 'Apple M2 Max',
 'Apple M2 Pro',
 'Apple M3

In [15]:
df[df['Processor_Model'] == 'AMD A Series']

,Brand,Laptop_Model_Name,Laptop_Model_Number,Processor_Brand,Processor_Model,Storage_Type,Operating_System,Display_Resolution,Extracted_Rating,Battery_Life(Hours_Upto),Price,Stock,Time_Of_Extraction,URL,Source,Storage_Capacity(GB),Display_Size(Inches),RAM(GB),No_Of_Reviews,Laptop_Dimensions,Laptop_Weight(Pounds),image_src
179,HP,HP Chromebook,16W64UT#ABA-CR,AMD,AMD A Series,SSD,Chrome OS,1366x768 Pixels,4.1,NaN,55.0,True,2024-12-05 12:33:00,https://www.amazon.com/HP-Chromebook-Education-A4-9120C-11-6-inch/dp/B08NHSPCR6/ref=sr_1_169?dib=eyJ2IjoiMSJ9.ADwoRjH1v5sRBTlFdPZ3U3J9Uz27D3Lr9OqT-aGcNzUWkIp2JfZ24IrgP6Y4iR9W2TMtXnHFrOgE5-_Kv8HAbtPJCLOUtBBEKwmCC_CppkHz3hJ4-dBRyQscme0Ixd69NoIbDapdN3Kpwuoodwmj14RNmiAs-6LbhYoERO0E4T3MX9aSaGr-4HGwbWv0QXD0uFjHHCakC5rVvK7v7AEl4wYUy0AA2bNEMVZbqNFALtZdA4qeT8wbu3Lnay19Zv_drCeBxSGUYw11ESR3-R2XH7fGx-WL7dAx1kZ2F87oXhs.GP1xMOLaNRT3rRcPKG0OHc8gUH6pf2VwCnQTikQR2ew&dib_tag=se&qid=1726997935&s=pc&sr=1-169,Amazon,32,16.0,4,565,9.10 x 9.10 x 9.10 inches,1.0,https://m.media-amazon.com/images/I/81LUKMXGZJL.__AC_SX300_SY300_QL70_FMwebp_.jpg
1462,HP,HP Chromebook,16W64UT#ABA-CR,AMD,AMD A Series,SSD,Chrome OS,1366x768 Pixels,4.1,NaN,55.0,True,2024-12-05 12:33:00,https://www.amazon.com/HP-Chromebook-Education-A4-9120C-11-6-inch/dp/B08NHSPCR6/ref=sr_1_169?dib=eyJ2IjoiMSJ9.ADwoRjH1v5sRBTlFdPZ3U3J9Uz27D3Lr9OqT-aGcNzUWkIp2JfZ24IrgP6Y4iR9W2TMtXnHFrOgE5-_Kv8HAbtPJCLOUtBBEKwmCC_CppkHz3hJ4-dBRyQscme0Ixd69NoIbDapdN3Kpwuoodwmj14RNmiAs-6LbhYoERO0E4T3MX9aSaGr-4HGwbWv0QXD0uFjHHCakC5rVvK7v7AEl4wYUy0AA2bNEMVZbqNFALtZdA4qeT8wbu3Lnay19Zv_drCeBxSGUYw11ESR3-R2XH7fGx-WL7dAx1kZ2F87oXhs.GP1xMOLaNRT3rRcPKG0OHc8gUH6pf2VwCnQTikQR2ew&dib_tag=se&qid=1726997935&s=pc&sr=1-169,Amazon,32,16.0,4,565,9.10 x 9.10 x 9.10 inches,1.0,https://m.media-amazon.com/images/I/81LUKMXGZJL.__AC_SX300_SY300_QL70_FMwebp_.jpg


In [9]:
processor_models = df['Processor_Model'].unique()

# Regular expression pattern to identify malformed processor names
nonepattern = re.compile(r'(\d+(\.\d+)?\s*(GHz|GHz\s*none|none|MHz|E\+?\d*)|[^A-Za-z0-9\s])')

# Find and print all models that match the pattern
malformed_models = [model for model in processor_models if model and nonepattern.search(str(model))]

malformed_models

['Intel Core Ultra 7 Series 1 Evo Edition',
 'Intel Core Ultra 7 Series 2 Evo Edition',
 'AMD A-Series A6']

In [27]:
df.columns

Index(['Brand', 'Laptop_Model_Name', 'Laptop_Model_Number', 'Processor_Brand',
       'Processor_Model', 'Storage_Type', 'Operating_System',
       'Display_Resolution', 'Extracted_Rating', 'Battery_Life(Hours_Upto)',
       'Price', 'Stock', 'Time_Of_Extraction', 'URL', 'Source',
       'Storage_Capacity(GB)', 'Display_Size(Inches)', 'RAM(GB)',
       'No_Of_Reviews', 'Laptop_Dimensions', 'Laptop_Weight(Pounds)',
       'image_src'],
      dtype='object')

In [34]:
df.shape

(4551, 22)

In [35]:
filtered_df = df[df['Processor_Model'] == 'Core i5']
filtered_df.shape

(1162, 22)

In [40]:
sfiltered_df = filtered_df[filtered_df['Storage_Capacity(GB)'] == 2048]
sfiltered_df

,Brand,Laptop_Model_Name,Laptop_Model_Number,Processor_Brand,Processor_Model,Storage_Type,Operating_System,Display_Resolution,Extracted_Rating,Battery_Life(Hours_Upto),Price,Stock,Time_Of_Extraction,URL,Source,Storage_Capacity(GB),Display_Size(Inches),RAM(GB),No_Of_Reviews,Laptop_Dimensions,Laptop_Weight(Pounds),image_src
8,HP,TPN-I139_767K9AV,TPN-I139_767K9AV,Intel,Core i5,SSD,Windows 11 Home,1600 x 900 Pixels,5.0,NaN,706.11,True,2024-12-05 12:16:53,https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyNTU2ODExODc2NTY2MDA3OjE3MjY5OTc5MTM6c3BfbXRmX2Jyb3dzZTozMDAyODg1NDU2MDMzMDI6OjA6Og&url=%2FHP-Premium-Touchscreen-i5-1335U-Backlit%2Fdp%2FB0CKJ552PH%2Fref%3Dsr_1_14_sspa%3Fdib%3DeyJ2IjoiMSJ9.Mxv-LfaT1mRTkqi6GWEFXxFggO64cMc5a5WQAxAoDYKDc12AZYR8P_ulvGvs2fWDJ7_Nm3Q_vhpmjYCsv0OJPJs6Bo1FRX66cFxFfjDS5M6onhimzcAeCOZ3ganbR1ztxCB3tNO3H2yyijUubD6xTB3G5UxB2MqPQQaHrdLyLai29xSPy1hZkKf5Sm2MjOm9tSgk53w2mGq_T8vokhTRQYuN1uCwbBymaj5IEXp_6tzKZ-DZ8lOcjCmWHFWVn2fkST3y58q3_y3AxTbeKQumI-hyzZmMa7tonKGyaYVia00.eFMZpiqa4BbfF8ul13G1oFWFR-jOJGFy_-1DtQHxgmY%26dib_tag%3Dse%26qid%3D1726997913%26s%3Dpc%26sr%3D1-14-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9tdGZfYnJvd3Nl%26psc%3D1,Amazon,2048,17.3,32,6,15.78 x 10.15 x 0.81 inches,4.600,https://m.media-amazon.com/images/I/611dsCsxGmL.__AC_SX300_SY300_QL70_FMwebp_.jpg
17,Dell,Dell Inspiron,Dell Laptop,Intel,Core i5,SSD,Windows 11 Pro,1920 x 1080 Pixels,4.1,6.5,469.00,True,2024-12-05 12:17:53,https://www.amazon.com/Dell-Laptop-Touchscreen-Quad-Core-i5-1135G7/dp/B0CC936WF1/ref=sr_1_25?dib=eyJ2IjoiMSJ9.Mxv-LfaT1mRTkqi6GWEFXxFggO64cMc5a5WQAxAoDYKDc12AZYR8P_ulvGvs2fWDJ7_Nm3Q_vhpmjYCsv0OJPJs6Bo1FRX66cFxFfjDS5M6onhimzcAeCOZ3ganbR1ztxCB3tNO3H2yyijUubD6xTB3G5UxB2MqPQQaHrdLyLai29xSPy1hZkKf5Sm2MjOm9tSgk53w2mGq_T8vokhTRQYuN1uCwbBymaj5IEXp_6tzKZ-DZ8lOcjCmWHFWVn2fkST3y58q3_y3AxTbeKQumI-hyzZmMa7tonKGyaYVia00.eFMZpiqa4BbfF8ul13G1oFWFR-jOJGFy_-1DtQHxgmY&dib_tag=se&qid=1726997913&s=pc&sr=1-25,Amazon,2048,15.6,16,214,14.11 x 9.27 x 0.83 inches,3.800,https://m.media-amazon.com/images/I/71gbrvRJI-L.__AC_SX300_SY300_QL70_FMwebp_.jpg
53,Dell,Latitude,5550,Intel,Core i5,SSD,Windows 11 Pro,1920 x 1080 Pixels,5.0,NaN,1240.00,True,2024-12-05 12:20:58,https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyOTM4MjAxNTQxMTI1MzkyOjE3MjY5OTc5MTc6c3BfYnRmX2Jyb3dzZTozMDAzODI1NjExMjY0MDI6OjA6Og&url=%2FDell-Latitude-5550-Business-Fingerprint%2Fdp%2FB0DFCMSWKR%2Fref%3Dsr_1_57_sspa%3Fdib%3DeyJ2IjoiMSJ9.3XJJxRRmtf7U-mXdsYFw-MSImixELglz2Bdcxa-DUbG9lhciUbUM_nJxeUMHJ_4U8WoXO4cwyqa8NFXuTNeXKUiQ8wmSfOqC5mkUTPMsl95Q0wZIALVQytM8q5R14rYEJLD0kTJJdCJ2QAiiVEuMpyJGJ-CGEPaBR9G1ALMcS2fxdF_H2P4NGGyuo-sFMgsBp6T8FUAlAmofVAlqXWTCMZ3CK4zEf7AGNrKwIejiURaCJHq377RGDlMbggQ-X3kw0CcZxcjoFK9n3UZ8Dc9ap6nbV-RU0_pKVtNmAbSRFOc.2CXWjMeriJGTZuAnQ1nV6k2J4ODdRrgiRcOh40ohd-k%26dib_tag%3Dse%26qid%3D1726997917%26s%3Dpc%26sr%3D1-57-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9idGZfYnJvd3Nl%26psc%3D1,Amazon,2048,15.6,16,7,14.10 x 9.40 x 0.82 inches,3.560,https://m.media-amazon.com/images/I/81hWo-897DL.__AC_SX300_SY300_QL70_FMwebp_.jpg
56,Lenovo,ThinkPad,T16,Intel,Core i5,SSD,Windows 11 Pro,1920 x 1200 Pixels,5.0,NaN,879.99,True,2024-12-05 12:21:11,https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozOTY2Mjc2MDU2MTEzNzg5OjE3MjY5OTc5MjE6c3BfYXRmX2Jyb3dzZTozMDA0MDg1NjAxNzkxMDI6OjA6Og&url=%2FLenovo-ThinkPad-T16-Business-Computer%2Fdp%2FB0DGGC8DCR%2Fref%3Dsr_1_51_sspa%3Fdib%3DeyJ2IjoiMSJ9._lsvuhAP0MqLdYMlQ2WVd5WiMcrEaU2GKe0ArwgEyuDDshwG-A4DyEH_Jceb4NPFYStz1DJ_YL3-BnPnpcFdx8_ImbGccQsEgG_dg9MAZky1lUKeqHOGmw9sdGpfpfTl4qVrRBKKTyk8TX2Sd0h4FKW1tw8zHD08nLQYlMZHOgSvdnPBp_8tCfonZk53iRk3N-H0oQuDY6PSyxh0Sbf-ETL67ab9fd-Sgcp6G7MJCkFCPUSgR1DFhUMWxCghGIpxOWyCmTeXmwa8R0yUa6uPu7HahpANq1hovVT24VUiirs.yPicvKOwJ4H9KENEA88a5cVBmQCYF4Wn6-4O6x9XgF4%26dib_tag%3Dse%26qid%3D1726997921%26s%3Dpc%26sr%3D1-51-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1,Amazon,2048,16.0,24,14,14.25 x 10.06 x 0.81 inches,3.900,https://m.media-amazon.com/images/I/71icfr-0ChL.__AC_SX300_SY300_QL70_FMwebp_.jpg
78,Dell,Inspiron,3511,Intel,Core i5,SSD,Windows 

In [37]:
asfiltered_df = sfiltered_df[sfiltered_df['RAM(GB)'] == 32]
asfiltered_df.

(62, 22)

In [38]:
df['RAM(GB)'].unique()

array([ 16,  32,   8,  40,   4,  20,   2,  64,  12,  24,   6, 128,  18,
        96,  48,  36])

In [39]:
l = list(df['RAM(GB)'].unique())
for i in l:
    print(type(i))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [41]:
df.columns

Index(['Brand', 'Laptop_Model_Name', 'Laptop_Model_Number', 'Processor_Brand',
       'Processor_Model', 'Storage_Type', 'Operating_System',
       'Display_Resolution', 'Extracted_Rating', 'Battery_Life(Hours_Upto)',
       'Price', 'Stock', 'Time_Of_Extraction', 'URL', 'Source',
       'Storage_Capacity(GB)', 'Display_Size(Inches)', 'RAM(GB)',
       'No_Of_Reviews', 'Laptop_Dimensions', 'Laptop_Weight(Pounds)',
       'image_src'],
      dtype='object')

In [43]:
df[df['Source'] == 'Amazon']['Processor_Brand'].unique()

array(['Intel', 'AMD', 'MediaTek', 'Apple', 'Alwinner', 'mtk', nan,
       'Qualcomm'], dtype=object)

In [60]:
df['Brand'].value_counts()

Brand
HP               937
Lenovo           759
Dell             570
MSI              521
Apple            395
ASUS             327
Acer             264
Microsoft        106
CHUWI             76
Zuleisy           70
SAIWAN            66
DELL              52
LG                31
SAMSUNG           30
TPV               28
Samsung           24
ACEMAGIC          22
Infinix           22
jumper            20
ZHAOHUIXIN        20
Razer             17
Alienware         14
ULTIMUS           13
GIGABYTE          13
Naclud            12
SGIN              10
ApoloSign         10
ist computers      8
Panasonic          8
AimCare            8
Thomson            6
NIAKUN             6
Oemgenuine         6
Molegar            6
Colorful           5
KOOFORWAY          4
FUNYET             4
ANPCOWER           4
ZENAERO            4
MICROSOFT          3
ZEBRONICS          3
VGKE               2
Akocrsiy           2
INHONLAP           2
Avita              2
AXL                2
Maxsignage         2
WIPEMIK

In [72]:
from collections import defaultdict
# Extract unique brands
brands = df['Brand'].dropna().unique()
# Dynamically create brand mapping using fuzzy matching (simple approach)
brand_mapping = defaultdict(str)

def normalize_brand_name(brand):
    # Check if the brand is not NaN
    if isinstance(brand, str):
        brand_lower = brand.lower()

        # Check for common brand patterns and group them
        for known_brand in brands:
            if known_brand.lower() in brand_lower:
                return known_brand  # Normalize to the known brand name

    return brand  # If no match, return the original bran

# Apply the normalization function to all brand names
df['Normalized_Brand'] = df['Brand'].apply(normalize_brand_name)
df['Normalized_Brand'].value_counts()

Normalized_Brand
HP               937
Lenovo           759
Dell             622
MSI              521
Apple            395
ASUS             327
Acer             264
Microsoft        109
CHUWI             76
Zuleisy           70
SAIWAN            66
SAMSUNG           54
LG                31
TPV               28
ACEMAGIC          22
Infinix           22
jumper            20
ZHAOHUIXIN        20
Razer             17
Alienware         14
ULTIMUS           13
GIGABYTE          13
Naclud            12
ApoloSign         10
SGIN              10
AimCare            8
Panasonic          8
ist computers      8
Thomson            6
NIAKUN             6
Molegar            6
Oemgenuine         6
Colorful           5
ZENAERO            4
FUNYET             4
ANPCOWER           4
KOOFORWAY          4
ZEBRONICS          3
Avita              2
AXL                2
ECOHERO            2
Primebook          2
Akocrsiy           2
KAIGERR            2
Gateway            2
Gina Joyfurno      2
Lapbook          

In [73]:
len(df['Normalized_Brand'].unique())

64

In [78]:
sorted(df['Normalized_Brand'].dropna().unique())

['ACEMAGIC',
 'ANPCOWER',
 'ASUS',
 'AXL',
 'Acer',
 'AimCare',
 'Akocrsiy',
 'Alienware',
 'ApoloSign',
 'Apple',
 'Avita',
 'BiTECOOL',
 'CHUWI',
 'Colorful',
 'Coolby',
 'Dell',
 'ECOHERO',
 'FUNYET',
 'GIGABYTE',
 'Gateway',
 'Getac',
 'Gina Joyfurno',
 'HP',
 'IJKKJI',
 'INHONLAP',
 'Infinix',
 'KAIGERR',
 'KOOFORWAY',
 'LETSUNG',
 'LG',
 'Lapbook',
 'Lenovo',
 'MSI',
 'Maxsignage',
 'Microsoft',
 'Molegar',
 'NIAKUN',
 'Naclud',
 'Nmybwo',
 'Oemgenuine',
 'Panasonic',
 'Primebook',
 'Razer',
 'SAINTDISE',
 'SAIWAN',
 'SAMSUNG',
 'SGIN',
 'THKGRCE',
 'TPV',
 'Thomson',
 'ULTIMUS',
 'VGKE',
 'Vaio',
 'WINGS',
 'WIPEMIK',
 'ZEBRONICS',
 'ZENAERO',
 'ZHAOHUIXIN',
 'Zuleisy',
 'ist computers',
 'jumper',
 'realme',
 'walker']

In [74]:
len(df['Brand'].unique())

67

In [64]:
df['Storage_Capacity(GB)'].unique()

array([    512,    2048,      64,     128,    8192,    4096,    1000,
           256,      32,     576,      16,       4,     250,     500,
           192,     320, 1048576,    6144,    1024,    2000,     480,
          1128,     750])

In [65]:
df[df['Storage_Capacity(GB)'] == 1048576]

,Brand,Laptop_Model_Name,Laptop_Model_Number,Processor_Brand,Processor_Model,Storage_Type,Operating_System,Display_Resolution,Extracted_Rating,Battery_Life(Hours_Upto),Price,Stock,Time_Of_Extraction,URL,Source,Storage_Capacity(GB),Display_Size(Inches),RAM(GB),No_Of_Reviews,Laptop_Dimensions,Laptop_Weight(Pounds),image_src
946,Lenovo,Lenovo,82H701FYUS,Intel,Core i5,SSD,Windows,1920 x 1080 Pixels,4.4,12.0,358.67,True,2024-12-05 14:50:58,https://www.amazon.com/Lenovo-IdeaPad-Laptop-i5-1135G7-Graphics/dp/B0B8JZ9WQM/ref=sr_1_868?dib=eyJ2IjoiMSJ9.VvHFFBJZmzErnjsS-JyGKDukxh_QxCDGLO3__EVVbXn0oxb2lHFx4UtDVeDMynrcxr2dh6pc0UQjBDZnP6bMBr7QnPwYxaGClFqSrs5i7P4WrWPawHEoqKoIOndN2LAnrVXzkQmDWB-tpZX4q2TLn30K75V1cySffagr3Wq1YgL7fHxumfkJLtV5Sftx7vIur-rc0vPpMd7nPqU4za8zu1RP7UXf7Oxd2W7jmcnklouT0nP3e1uFUMgoZw06im9jLuJBVjrmJVbC8fhMC15LoRQ_jPbWrwjvqsZwEw5vC7g.6es9Yqo9nckNn1ykn_tcCTAGqmEkVySiy3LUoibaNOA&dib_tag=se&qid=1726998990&s=pc&sr=1-868,Amazon,1048576,14.0,8,37,12.88 x 9.49 x 0.78 inches,3.08,https://m.media-amazon.com/images/I/61F9nQbmH7L.__AC_SX300_SY300_QL70_FMwebp_.jpg
1195,HP,Essential,Essential,AMD,Ryzen 5,SSD,Windows 11 Pro,1920 x 1080 Pixels,4.0,NaN,699.00,True,2024-12-05 15:10:42,https://www.amazon.com/HP-Storage-Processor-i7-1165G7-Ethernet/dp/B0DFHQNDLB/ref=sr_1_1066?dib=eyJ2IjoiMSJ9.LSAkUfmn_WZku7KtfYFSclNSaS7SFvW9Kk0Oa8Mv1_1vqNpxDuCsNsHTidS03ubhYwOnMj7RBDj-a2oU9JsHi7MLksZlqwNLJzSuvAMYZ1ROcumI8Fp5yCML13Jv1SY90sdjEbzuuhYDFA6daOcUFBBF-1MGzddOgPcRjA0fSDirNJKYEqB5xlFSOl-mWYfTid49WU1M-jdBVCT_kBQlkgqtJYEsQ2_ny6IKkNjNsYh3eXU-jVV8oGF5mRTWBoeztnXwg19F1JJ3AqRPP61wjA0J8sUyPvNBFZicoxq1Fg4.yF-qoSx7vvanEiGYKqREwjM1BoFmAzcY0SJ472XlYGk&dib_tag=se&qid=1726999029&s=pc&sr=1-1066,Amazon,1048576,15.6,32,3,14.12 x 9.53 x 0.70 inches,3.73,https://m.media-amazon.com/images/I/71h+1x1mhUL._AC_SY300_SX300_.jpg
2229,Lenovo,Lenovo,82H701FYUS,Intel,Core i5,SSD,Windows,1920 x 1080 Pixels,4.4,12.0,358.67,True,2024-12-05 14:50:58,https://www.amazon.com/Lenovo-IdeaPad-Laptop-i5-1135G7-Graphics/dp/B0B8JZ9WQM/ref=sr_1_868?dib=eyJ2IjoiMSJ9.VvHFFBJZmzErnjsS-JyGKDukxh_QxCDGLO3__EVVbXn0oxb2lHFx4UtDVeDMynrcxr2dh6pc0UQjBDZnP6bMBr7QnPwYxaGClFqSrs5i7P4WrWPawHEoqKoIOndN2LAnrVXzkQmDWB-tpZX4q2TLn30K75V1cySffagr3Wq1YgL7fHxumfkJLtV5Sftx7vIur-rc0vPpMd7nPqU4za8zu1RP7UXf7Oxd2W7jmcnklouT0nP3e1uFUMgoZw06im9jLuJBVjrmJVbC8fhMC15LoRQ_jPbWrwjvqsZwEw5vC7g.6es9Yqo9nckNn1ykn_tcCTAGqmEkVySiy3LUoibaNOA&dib_tag=se&qid=1726998990&s=pc&sr=1-868,Amazon,1048576,14.0,8,37,12.88 x 9.49 x 0.78 inches,3.08,https://m.media-amazon.com/images/I/61F9nQbmH7L.__AC_SX300_SY300_QL70_FMwebp_.jpg
2478,HP,Essential,Essential,AMD,Ryzen 5,SSD,Windows 11 Pro,1920 x 1080 Pixels,4.0,NaN,699.00,True,2024-12-05 15:10:42,https://www.amazon.com/HP-Storage-Processor-i7-1165G7-Ethernet/dp/B0DFHQNDLB/ref=sr_1_1066?dib=eyJ2IjoiMSJ9.LSAkUfmn_WZku7KtfYFSclNSaS7SFvW9Kk0Oa8Mv1_1vqNpxDuCsNsHTidS03ubhYwOnMj7RBDj-a2oU9JsHi7MLksZlqwNLJzSuvAMYZ1ROcumI8Fp5yCML13Jv1SY90sdjEbzuuhYDFA6daOcUFBBF-1MGzddOgPcRjA0fSDirNJKYEqB5xlFSOl-mWYfTid49WU1M-jdBVCT_kBQlkgqtJYEsQ2_ny6IKkNjNsYh3eXU-jVV8oGF5mRTWBoeztnXwg19F1JJ3AqRPP61wjA0J8sUyPvNBFZicoxq1Fg4.yF-qoSx7vvanEiGYKqREwjM1BoFmAzcY0SJ472XlYGk&dib_tag=se&qid=1726999029&s=pc&sr=1-1066,Amazon,1048576,15.6,32,3,14.12 x 9.53 x 0.70 inches,3.73,https://m.media-amazon.com/images/I/71h+1x1mhUL._AC_SY300_SX300_.jpg


In [79]:
df.columns

Index(['Brand', 'Laptop_Model_Name', 'Laptop_Model_Number', 'Processor_Brand',
       'Processor_Model', 'Storage_Type', 'Operating_System',
       'Display_Resolution', 'Extracted_Rating', 'Battery_Life(Hours_Upto)',
       'Price', 'Stock', 'Time_Of_Extraction', 'URL', 'Source',
       'Storage_Capacity(GB)', 'Display_Size(Inches)', 'RAM(GB)',
       'No_Of_Reviews', 'Laptop_Dimensions', 'Laptop_Weight(Pounds)',
       'image_src', 'Normalized_Brand'],
      dtype='object')

In [81]:
df['Extracted_Rating'].unique()

array([   52,     6,   878,   639,   473,   141,   203,     3,    13,
         940,   657,   823,   892,   329,   535,    54,   214,   729,
         882,   383,    63,    14,     5,     2,     8,    48,   390,
         493,    58,   395,   312,   145,   496,   470,   954,   780,
           0,    15,   311,   520,   105,   688,    32,   482,   116,
         232,   762,     7,   463,    43,   386,   318,    41,   122,
         702,   551,    37,    65,   270,   385,    35,   593,   225,
         158,    87,   600,   443,   740,   156,    53,   302,   573,
         650,   100,   652,    49,    25,   120,    89,    76,   462,
          17,   958,     1,   211,   118,    44,   127,   172,   638,
          33,   215,   923,   714,    86,   636,    68,   103,   132,
         431,     4,    75,   409,   640,    72,    31,   405,   732,
         899,   256,   237,   284,    16,   352,   106,    21,   603,
          92,   654,    10,   155,   565,   591,   223,   204,    27,
         687,   152,

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4551 entries, 0 to 4550
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Brand                     4550 non-null   object 
 1   Laptop_Model_Name         4519 non-null   object 
 2   Laptop_Model_Number       4307 non-null   object 
 3   Processor_Brand           4547 non-null   object 
 4   Processor_Model           4551 non-null   object 
 5   Storage_Type              4551 non-null   object 
 6   Operating_System          4551 non-null   object 
 7   Display_Resolution        4543 non-null   object 
 8   Extracted_Rating          4056 non-null   float64
 9   Battery_Life(Hours_Upto)  1217 non-null   float64
 10  Price                     4551 non-null   float64
 11  Stock                     4551 non-null   bool   
 12  Time_Of_Extraction        4551 non-null   object 
 13  URL                       4551 non-null   object 
 14  Source  